# Precinct Matching Framework

In [1]:
import pandas as pd
import geopandas as gpd
from pprint import pprint

### Import the datasets

In [2]:
county_id = 'Orleans County'

county_results_filename = 'election_results_county_id={}.csv'.format(county_id)
county_results_df = pd.read_csv(county_results_filename)
county_shapefile_filename = 'shapefile_county_id={}'.format(county_id)
county_shapefile_gdf = gpd.read_file(county_shapefile_filename)

# correct for the truncation caused by 10 character column name limit in shapefiles
county_shapefile_gdf.rename(columns={'original_p':'original_precinct_name'}, inplace=True)

The next cell aliases `county_results_df` as `df` and `county_shapefile_gdf` as `gdf` here because typing fewer characters allows for faster data exploration. 

In [3]:
df = county_results_df.copy()
gdf = county_shapefile_gdf.copy()

In [4]:
df.head()

,county,precinct,HOR_DEM,HOR_REP,HOR_CON,HOR_GRE,HOR_WOR,HOR_IND,HOR_WEP,HOR_REF,...,Gov_CON,Gov_GRE,Gov_WOR,Gov_IND,Gov_WEP,Gov_REF,Gov_SAM,Gov_LBT,county_id,original_precinct_name
0,Orleans County,Town of Albion 1 LD 3,99,98,13,0,13,0,9,0,...,15,2,3,1,0,4,0,11,Orleans County,Town of Albion 1 LD 3
1,Orleans County,Town of Albion 2 LD 3,179,121,27,0,12,3,2,7,...,31,6,3,1,1,2,5,25,Orleans County,Town of Albion 2 LD 3
2,Orleans County,Town of Albion 3 LD 3,124,101,12,0,7,2,1,6,...,15,3,0,5,0,1,2,17,Orleans County,Town of Albion 3 LD 3
3,Orleans County,Town of Albion 4 LD 3,112,58,11,0,6,2,1,6,...,17,2,1,4,0,1,4,4,Orleans County,Town of Albion 4 LD 3
4,Orleans County,Town of Albion 5 LD 3,83,132,16,0,7,2,1,6,...,15,1,0,2,0,2,3,15,Orleans County,Town of Albion 5 LD 3


In [5]:
gdf.head()

,STATEFP,COUNTYFP,VTDST,NAMELSAD,VTDI,LSAD,CHNG_TYPE,ORIG_NAME,ORIG_CODE,RELATE,NAME,VINTAGE,FUNCSTAT,JUSTIFY,MTFCC,county_id,original_precinct_name,geometry
0,36,073,000001,000001,A,00,None,None,None,None,000001,90,N,None,G5240,Orleans County,000001,"POLYGON Z ((-78.19374 43.24656 0.00000, -78.19..."
1,36,073,000002,000002,A,00,None,None,None,None,000002,90,N,None,G5240,Orleans County,000002,"POLYGON Z ((-78.19404 43.23445 0.00000, -78.19..."
2,36,073,000003,000003,A,00,None,None,None,None,000003,90,N,None,G5240,Orleans County,000003,"POLYGON Z ((-78.21033 43.24312 0.00000, -78.21..."
3,36,073,000004,000004,A,00,None,None,None,None,000004,90,N,None,G5240,Orleans County,000004,"POLYGON Z ((-78.20881 43.25094 0.00000, -78.20..."
4,36,073,000008,000008,A,00,None,None,None,None,000008,90,N,None,G5240,Orleans County,000008,"POLYGON Z ((-78.31050 43.13145 0.00000, -78.31..."


In [23]:
gdf.shape

(40, 18)

In [24]:
gdf.VTDST.nunique()

40

### Check Preconditions
These should all pass - they're here to ensure that everything in `setup.ipynb` worked correctly

In [6]:
# TODO: Pass the precondition described above which takes the form of an assert statement in this cell.
assert 'county_id' in df.columns and 'county_id' in gdf.columns
assert 'original_precinct_name' in df.columns and 'original_precinct_name' in gdf.columns

### Get Polling locations

1. Download `matching/Orleans County/Poll-Site-List-2020.pdf` from https://orleanscountyny.com/wp-content/uploads/2020/06/Poll-Site-List-2020.pdf on 2021-01-21.
2. Convert to `matching/Orleans County/Poll-Site-List-2020.csv` with https://pdftables.com/upload/view/lfm9kv9ll51xok8vydewdy1d4/Poll-Site-List-2020.pdf on 2021-01-21.
3. Import below:

In [22]:
poll_df = pd.read_csv('Poll-Site-List-2020.csv')
poll_df['State'] = 'New York'
poll_df.head()

,Orleans County Poll Site Locations,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,State
0,Town,ED,Poll Site,Address,City,New York
1,Albion,1,Hoag Library,134 S Main St,Albion,New York
2,Albion,2,Hoag Library,134 S Main St,Albion,New York
3,Albion,3,Hoag Library,134 S Main St,Albion,New York
4,Albion,4,Hoag Library,134 S Main St,Albion,New York


In [20]:
poll_df.columns

Index(['Town', 'ED', 'Poll Site', 'Address', 'City', 'State'], dtype='object')

In [21]:
poll_df.to_csv('Poll-Site-List-2020-.csv')

Now I will geocode these addresses into precinct geometries

In [15]:
import censusgeocode as cg

In [19]:
cg.addressbatch('Poll-Site-List-2020-indexed.csv')

[OrderedDict([('id', ''),
              ('address', 'Town, ED, Poll Site, Address'),
              ('match', False),
              ('matchtype', None),
              ('parsed', None),
              ('tigerlineid', None),
              ('side', None),
              ('statefp', None),
              ('countyfp', None),
              ('tract', None),
              ('block', None),
              ('lat', None),
              ('lon', None)]),
 OrderedDict([('id', '22'),
              ('address',
               'Murray, 2.0, Murray Town Hall/Highway, 3840 Fancher Rd'),
              ('match', False),
              ('matchtype', None),
              ('parsed', None),
              ('tigerlineid', None),
              ('side', None),
              ('statefp', None),
              ('countyfp', None),
              ('tract', None),
              ('block', None),
              ('lat', None),
              ('lon', None)]),
 OrderedDict([('id', '23'),
              ('address',
               'Murray,

### General Modifications
Its normally benificial to apply some modifications uniformly to all precincts. For example, its good practice to make everything lower case. This modification is made in `edit_precinct_name` - read its specification to learn more about how to use it to make more modifications. Consider removing substrings that appear in every precinct name like `voting district`. Be careful of removing words that will result in duplicate precinct names. For example, if there are two `Newtown Voting District` precincts in the shapefile, and the election results have `Newtown Boro` and `Newtown Township`, its okay to remove `Voting District`, but you probably don't want to remove `Boro` or `Township`. Of course, this will differ from County to County - so be vigilant!

In [7]:
dataset_name_df = 'df'
dataset_name_gdf = 'gdf'

def edit_precinct_name(prec_name, 
    remove_lst=[], 
    target_to_replacement={},
    stopping_words=[],
    prec_dict={}):
    '''
    Returns a lower case precinct name (string) with certian modifications depending other arguments. 
    
    Modifications are performed in order of the parameters they depend on. By convention, case is 
    ignored by making prec_name lower case. Accordingly, one should pass arguements with lower case
    elements. That is, keys of the dictionaries and elements of lists should be lower case strings.

	Parameters:
		prec_name (str): precinct name
		remove_lst ((str) list): if a string in this list is a substring in prec_name it will be removed. 
            All elements should be lower case.
        target_to_replacement ({str:str} dictionary): keys (targets) will be replaced with their 
            corresponding value (replacements) in prec_name. All keys should be lower case.
        stopping_words ({str} list): If any substring of prec_name contains a element of stopping_words
             that is adjacent to a space character it will be removed. All elements should be lower case.
        prec_dict ({str:str} dictionary): After all the modifications above, if the edited prec_name
            string is in the set of keys for prec_dict, then it will be replaced with that key's value.  
            All keys should be lower case.

	Returns:
		prec_name (str): prec_name arguement returned with the 
    '''
    prec_name = str(prec_name).lower()
    for word in remove_lst:
        prec_name = prec_name.replace(word, '')
    for target, replacement in target_to_replacement.items():
        prec_name = prec_name.replace(target, replacement)
    words = prec_name.split()
    words = [word.lstrip('0') for word in words if word not in stopping_words]
    prec_name = " ".join(words)
    return prec_dict[prec_name] if prec_name in prec_dict.keys() else prec_name

df_to_gdf = {
}

gdf_to_df = {
}

# Tune the matching by adding optional arguements to edit
df['edited_precinct_name'] = df['original_precinct_name'].apply(lambda name: edit_precinct_name(name,prec_dict=df_to_gdf))
gdf['edited_precinct_name'] = gdf['original_precinct_name'].apply(lambda name: edit_precinct_name(name,prec_dict=gdf_to_df))

######## Manual Corrections ###########
# Make precinct specific corrections here like splitting one precinct into two because of new congressional districts
# e.g gdf.loc[gdf['VTDST']=='000290','edited_precinct_name'] = 'howard township'

# make the 'original_precinct_name, edited_precinct_name' for use in the loop below
df['original_precinct_name, edited_precinct_name'] = df[['original_precinct_name','edited_precinct_name']].apply(tuple, axis=1)
gdf['original_precinct_name, edited_precinct_name'] = gdf[['original_precinct_name','edited_precinct_name']].apply(tuple, axis=1)

######## Matching Framework ###########
unmatched_precinct_lst_gdf = []
unmatched_precinct_lst_df = []

precinct_list_df = sorted(list(df[df['county_id'] == county_id]['original_precinct_name, edited_precinct_name'].unique()), key=lambda x: x[1])
precinct_list_gdf = sorted(list(gdf[gdf['county_id'] == county_id]['original_precinct_name, edited_precinct_name'].unique()), key=lambda x: x[1])

precinct_set_df = {x[1] for x in precinct_list_df if x[1] not in unmatched_precinct_lst_df}
precinct_set_gdf = {x[1] for x in precinct_list_gdf if x[1] not in unmatched_precinct_lst_gdf}

unmatched_precincts_df = sorted(list(precinct_set_df - precinct_set_gdf))
unmatched_precincts_gdf = sorted(list(precinct_set_gdf - precinct_set_df))
n_unmatched = len(unmatched_precincts_df) + len(unmatched_precincts_gdf)
if n_unmatched > 0:
        print("county_id: '{}' | {} precincts in {} | {} precincts in {}:\n".format(county_id, len(precinct_list_df), dataset_name_df, len(precinct_list_gdf), dataset_name_gdf))
        n_precincts_total = len(precinct_list_df) + len(precinct_list_gdf)
        print(n_unmatched, " precincts are unmatched out of ", n_precincts_total)
        df_unmatched = df[(df['edited_precinct_name'].isin(unmatched_precincts_df)) & (df.county_id == county_id)]
        gdf_unmatched = gdf[(gdf['edited_precinct_name'].isin(unmatched_precincts_gdf)) & (gdf.county_id == county_id)]
        if n_unmatched > 100:
            print("\nLook for parterns and use change the parameters to edit_precinct_name accordingly.\n")
            for index, (original_precinct_name_df, edited_precinct_name_df) in enumerate(precinct_list_df):
                original_precinct_name_gdf, edited_precinct_name_gdf = precinct_list_gdf[index]
                if edited_precinct_name_df in unmatched_precincts_df and edited_precinct_name_gdf in unmatched_precincts_gdf:
                    print("{} <-- {} ({})".format(edited_precinct_name_df, original_precinct_name_df, dataset_name_df))
                    print("{} <-- {} ({})\n".format(edited_precinct_name_gdf, original_precinct_name_gdf, dataset_name_gdf))
        else:
            print("unmatched_precincts_df ({}) - len = {}| '{}':".format(dataset_name_df, len(unmatched_precincts_df), county_id), unmatched_precincts_df)
            print("\nunmatched_precincts_gdf ({}) - len = {}| '{}':".format(dataset_name_gdf, len(unmatched_precincts_gdf), county_id), unmatched_precincts_gdf)
            precinct_modification_dictionary_df_to_gdf = {unmatched_precincts_df[i]: unmatched_precincts_gdf[i] if i < len(unmatched_precincts_gdf) else '' for i in range(len(unmatched_precincts_df))}
            precinct_modification_dicitonary_gdf_to_df = {unmatched_precincts_gdf[i]: unmatched_precincts_df[i] if i < len(unmatched_precincts_df) else '' for i in range(len(unmatched_precincts_gdf))}
            print("{}  to {} precinct modification dictionary: ".format(dataset_name_df, dataset_name_gdf))
            print("'{}':".format(county_id))
            pprint(precinct_modification_dictionary_df_to_gdf)
            print("{}  to {} precinct modification dictionary: ".format(dataset_name_gdf, dataset_name_df))
            print("'{}':".format(county_id))
            pprint(precinct_modification_dicitonary_gdf_to_df)
            for index, (original_precinct_name_df, edited_precinct_name_df) in enumerate(precinct_list_df):
                original_precinct_name_gdf, edited_precinct_name_gdf = precinct_list_gdf[index]
                if edited_precinct_name_df in unmatched_precincts_df or edited_precinct_name_gdf in unmatched_precincts_gdf:
                    print("{} <-- {} ({})".format(edited_precinct_name_df, original_precinct_name_df, dataset_name_df))
                    print("{} <-- {} ({})\n".format(edited_precinct_name_gdf, original_precinct_name_gdf, dataset_name_gdf))
            else:
                print("Add unmatched precincts to the unmatched precinct.")
else:
    print("All Done! (make sure you have one to one matches)")

county_id: 'Orleans County' | 40 precincts in df | 40 precincts in gdf:

80  precincts are unmatched out of  80
unmatched_precincts_df (df) - len = 40| 'Orleans County': ['town of albion 1 ld 3', 'town of albion 2 ld 3', 'town of albion 3 ld 3', 'town of albion 4 ld 3', 'town of albion 5 ld 3', 'town of albion 6 ld 3', 'town of barre 1 ld 1', 'town of barre 2 ld 1', 'town of carlton 1 ld 4', 'town of carlton 2 ld 4', 'town of carlton 3 ld 4', 'town of clarendon 1 ld 1', 'town of clarendon 2 ld 1', 'town of clarendon 3 ld 1', 'town of gaines 1 ld 3', 'town of gaines 2 ld 3', 'town of gaines 3 ld 3', 'town of gaines 4 ld 3', 'town of kendall 1 ld 4', 'town of kendall 2 ld 4', 'town of kendall 3 ld 4', 'town of murray 1 ld 4', 'town of murray 2 ld 4', 'town of murray 3 ld 4', 'town of murray 4 ld 4', 'town of murray 5 ld 4', 'town of murray 6 ld 4', 'town of ridgeway 1 ld 2', 'town of ridgeway 2 ld 2', 'town of ridgeway 3 ld 2', 'town of ridgeway 4 ld 2', 'town of ridgeway 5 ld 2', 'town 